The idea is:
- for every landcover, create a destination folder
- split the landcover in tiles as one array is 188Gb
- at each split, there is a csv with the landcover values. Iterate it and assign 1 value to the corresopnding class and 0 to the rest
- create a subfolder with the content and name of the class
- merge all together

In [21]:
import os
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import pandas as pd
import math
import platform
from shapely.geometry import box
from matplotlib import pyplot as plt

In [2]:
# rootdir = os.path.dirname(os.path.abspath(__file__))

# landcover_directory = r"\\akif.internal\public\z_resources\im-wb\landcove_layers"
landcover_directory = r"C:\Users\admin\Downloads\landcover_layers"
landcover_classes_csv = "./tmp/landcover_classes.csv" #this works

In [3]:
def get_raster_data(path):
    file_list = []
    for file in os.listdir(path):
        # Iterate over all the files in the specified directory.
        if ".tif" in file:
            # Process the file if it has a .tif format.
            if platform.system() is "Windows":
                address = os.path.join(path, file).replace("/","\\")
            else:
                address = os.path.join(path, file).replace("\\","/")
                #build the path according the OS running the script

            if address not in file_list:
                # Add the file address to the list if it had not been added before.
                file_list.append(address)
        else:
            pass
    
    return file_list

landcover_list = get_raster_data(landcover_directory)



In [26]:
"""starting of the reading"""
clc_df = pd.read_csv(landcover_classes_csv) #this works
for raster_file in landcover_list[0:1]:
    #create a folder for the outputs
    
    """Create and go the folder which will contain the landcover reclass"""
    raster_directory = raster_file.replace('.tif','')
    output_dir = raster_directory + "_reclass"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    

In [22]:
"""read the landcover csv and select the value and the class"""
for row_index, row in clc_df.loc[3:3].iterrows():
        print(row_index)  
        # Load the raster file.
        # landcover_file = rasterio.open(raster_file)
        # landcover_array = landcover_file.read() #[z,x,y]
        # height = landcover_array.shape[1]
        # width  = landcover_array.shape[2]
        
        #create a copy
        # landcover_reclass = landcover_file.copy()
        
        # # do a reclass of the file
        # landcover_reclass[np.where(landcover_reclass = row['VALUE'] )] = 1
        # landcover_reclass[np.where(landcover_reclass != row['VALUE'] )] = 0
        
        # ISSUE: we get memory issues at doing this part. We have to consider how to solve this
     

3


In [34]:
"""try the same but with a gdal command with the calculator. I think It will be faster"""
import sys
sys.path.insert(1, './tmp')
import gdal_calc
## Raster Calculator##
gdal_calc_path = "./tmp/gdal_calc.py"

# Arguements.
input_file_path = raster_file
raster_name = os.path.basename(raster_file).replace('.tif','')
output_file_path = os.path.join(output_dir, raster_name + "_" + str(row['VALUE']) + ".tif")
calc_expr = "(A == {0})*1 + (A > {0})*(A < {0})*0".format(row['VALUE'])

# Generate string of process.
# gdal_calc_str = 'python {0} -A {1} --outfile={2} --calc={3} --NoDataValue=0 -co COMPRESS=DEFLATE -co TILED=YES'
# gdal_calc_process = gdal_calc_str.format(gdal_calc_path, input_file_path, output_file_path, calc_expr)

# """gdal_calc -A cci-lc-300m-2000.tif --outfile=cci-lc-300m-2000_Cropland_rainfed.tif --calc="(A==10)*1 + (A>10)*(A<10)*0" --NoDataValue=0"""
# print(gdal_calc_process)
# Call process.
# os.system(gdal_calc_process)

calculated_tiff = gdal_calc.Calc(calc_expr, A=input_file_path, outfile=output_file_path, NoDataValue=0, creation_options=["COMPRESS=DEFLATE", "TILED=YES"])

#Calc(calc="A*(A>0)", A="input.tif", A_Band=2, outfile="result.tif", NoDataValue=0)
# "Calc("A+B", A="input1.tif", B="input2.tif", outfile="result.tif")"
#creation_options=["COMPRESS=DEFLATE", "TILED=YES"]
calculated_tiff = None


AttributeError: 'NoneType' object has no attribute 'RasterXSize'

In [19]:
Calc(calc, outfile, NoDataValue=None, type=None, format=None, creation_options=[], allBands='', overwrite=False, debug=False, quiet=False, **input_files)

In [ ]:
os. getcwd()

'c:\\Users\\admin\\Documents\\01_Ruben_Scripts\\im.nca.postprocessing\\aggregation.region.classified'

In [ ]:
print(gdal_calc_process)

python ./tmp/gdal_calc.py -A Z:\z_resources\im-wb\landcover_layers\cci-lc-300m-2000.tif --outfile=Z:\z_resources\im-wb\landcover_layers\cci-lc-300m-2000\30.tif --calc=(A == 30)*1 + (A > 30)*(A < 30)*0 --NoDataValue=0 -co COMPRESS=DEFLATE -co TILED=YES


In [ ]:
"""Here we start tiling"""
tilesize = 1000
for i in range(0, width, tilesize)[0:1]: # Tilesize marks from where to where in width.
    for j in range(0, height, tilesize)[0:3]:
        # This is for the edge parts, so we don't get nodata from the borders.
        w0 = i # Start of width the array.
        w_plus = min(i+tilesize, width) - i # Addition value of width.
        w1 = w0 + w_plus # End of the array.
        h0 = j # start of height array
        h_plus = min(j+tilesize, height) - j # Addition value of heigth.
        h1 = h0 + h_plus
        
        """Create the geometry"""
        cols, rows = np.meshgrid(np.arange(w0, w1), np.arange(h0, h1))
        xs, ys = rasterio.transform.xy(landcover_file.transform, rows, cols)
        
        bbox = box(w0, h0, w1, h1)
        # geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=raster_image.crs)
        geo_row = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=landcover_file.crs)
        coords = gpd.GeoSeries(geo_row['geometry'])
        
        """Mask the data"""
        # out_image, out_transform = rasterio.mask.mask(landcover_file, coords, crop=True)

In [ ]:
geo_row.to_csv(landcover_directory + "/bbox_geoemtry_3" + ".csv")

In [ ]:
plt.plot(out_image[0,:,:])

In [ ]:
def reclassifier(landcover_layer):
    landcover_layer[np.where(landcover_layer = row['VALUE'] )] = 1
    landcover_layer[np.where(landcover_layer != row['VALUE'] )] = 0
    
    return landcover_layer

In [ ]:
#save the result
with rasterio.open(os.path.join(output_dir, data_name, '_', row['VALUE'],'.tif').replace("/","\\"), 'w', 
                    driver = landcover_file.driver, 
                    height = landcover_file.height, 
                    width = landcover_file.width, 
                    count = landcover_file.count, 
                    dtype = landcover_file.dtype, 
                    crs = landcover_file.crs, 
                    transform = landcover_file.transform, 
                    compress='deflate') as dst:
    
    dst.write(out_image)